#### basic_mind
    superviskeyed learning（教師あり）: regression（回帰）classification（分類）
        keyword: 特徴量、正解データ
    unsupervised learning（教師なし）: clustering（クラスタリング）dimensionality reduction（次元削減）
    
#### 各関数：
df["].value_counts() 1列分のデータ別個数の集計
df["].unique() 1列分のデータを重複を除いて抽出
df.dropna(how="all" or "any", axis=0:row or 1:col)　None削除
df.drop(index or colname, axis= 0:row or 1:col) 行またはカラム名指定削除
df.loc[index, col] 特定の行列の抽出 :スライサOK
df["].fillna("replaceValue")欠損値補完
df.loc[:,from:to] インデックス、カラム名指定取得
df_none = df[df.isnull().any(axis=1)] None行の取得
df_if = df[(df[a]>0) & (df[y]=="val")] 行の条件取得

df.sharp　データフレームの全体概要出力
df.columns カラム名取得


pd.concat([df_A, df_B], axis=0:addRow or 1:addCol, how=?)
pd.merge

#### 描画
df.plot(kind="scatter", x=val, y=val) 散布図


#### 機械学習
pd.get_dummies(dataframe, drop_first=True) ダミーデータへ変換

with open("fileName",wb) as f; pickle.dump("saveModel",f) モデル保存
with open("fileName",rb) as f; pickle.load("saveModel",f)　モデル読み込み

from sklearn.model_selection import train_test_split データの分割 複数可
x1,x2, y1,y2 = train_test_split(df_x, df_y, test_size=0.2, random_state=0)

from sklearn import tree 決定木 分類 class_weightは任意
model = tree.DecisionTreeClassifier(max_depth=depth, random_state=0, class_weight="balanced")
model.fit(x,y)
score = model.score(x=x_tr, y=y_tr)

from sklearn.linear_model import LinearRegression　回帰
model = LinearRegression()
model.fit(x,y) x:train y=corect
pred_value = model.predict(x)


# 機械学習ラーニング　CHAPTER9　bank.csv

## 文字列データのダミー化

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv("sukkiri-ml2-codes/datafiles/bank.csv")

colname = ["job","marital","education","default","housing","loan","contact"]
df_str = df.loc[:,colname]
df_dummy = pd.get_dummies(df_str, drop_first=True)
# Drop the original categorical columns from df
df_without_cat = df.drop(colname, axis=1)
# Concatenate the numerical columns with dummy variables horizontally
df2 = pd.concat([df_without_cat, df_dummy], axis=1)

## 訓練データとテストデータの分割、不要な項目を削除

In [7]:
drop_col = df2.drop(["id","age","amount","day","month"],axis=1)
train_val, test = train_test_split(drop_col,test_size=0.2,random_state=0)


## 機械学習による予測値の設定

In [ ]:
# 機械学習による予測値
from sklearn.linear_model import LinearRegression
pred_tr_val = train_val.dropna()
# need to predict
pred_x = train_val[train_val.isnull().any(axis=1)]
x = pred_tr_val.drop(["duration"],axis=1)
y = pred_tr_val["duration"]
model = LinearRegression()
model.fit(x,y)
pred_x = pred_x.dropna(axis=1)
pred_y = model.predict(pred_x)
train_val.loc[train_val["duration"].isnull(), "duration"] = pred_y



## 性能テスト

In [ ]:
from sklearn import tree

def leran(x,t,depth=1): # 訓練データと検証データのスコアを返す処理
    x_tr, x_test, y_tr, y_test = train_test_split(x,t,test_size=2, random_state=0)
    model = tree.DecisionTreeClassifier(max_depth=depth, random_state=0, class_weight="balanced")
    model.fit(x,y)
    score = model.score(x=x_tr, y=y_tr)
    score2 = model.score(x=x_test, y=y_test)
    return round(score,3), round(score2,3), model


for i in range(1,15):
    tr_score, test_score, model = learn(x,x,i)
    print(f"深さ{i}：訓練スコア{tr_score} テストスコア{test_score}")





## マハラノビスによる外れ値の除去

In [ ]:
# from sklearn.covariance import MinCovDet
# mcd = MinCovDet(random_state=0, support_fraction=0.7)
# mcd.fit(train_val)
# distance = mcd.mahalanobis(train_val)
# s_distance = pd.Series(distance) 
# tmp=s_distance.describe()

# iqr = tmp["75%"] - tmp["25%"] #IRQ
# maxval = (iqr*1.5) + tmp["75%"]
# minval = tmp["25%"] - (iqr*1.5)

# clear_data = s_distance[(s_distance > maxval) &(minval > s_distance)]
# clear_data # 該当データなし


In [ ]:

# 1. 相関係数を計算
# yと他の特徴量との相関係数を計算し、絶対値で降順にソート
correlations = train_val.corr()['y'].abs().sort_values(ascending=False)
print("相関係数（上位10項目）:")
print(correlations[1:11])  # yとの相関を除いて上位10項目を表示

# 2. より詳細な分析のために、特徴量とyの関係を可視化
import matplotlib.pyplot as plt
import seaborn as sns

# 相関係数上位5項目についてボックスプロットを作成
top_features = correlations[1:6].index
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()


for idx, feature in enumerate(top_features):
    sns.boxplot(data=train_val, x='y', y=feature, ax=axes[idx])
    axes[idx].set_title(f'{feature} vs y')



plt.tight_layout()
plt.show()